In [5]:
from ipycanvas import RoughCanvas
canvas = RoughCanvas()

canvas.stroke_rect(100, 100, 100, 100)
canvas.fill_rect(50, 50, 100, 100)

canvas.stroke_circle(300, 300, 100)
canvas.fill_circle(350, 350, 100)

canvas.stroke_line(200, 200, 300, 300)

canvas


RoughCanvas()

In [54]:
import numpy as np
from ipywidgets import interact
from ipycanvas import hold_canvas

canvas = RoughCanvas()

width, height = 320, 200

x, y = np.meshgrid(np.linspace(0, width, 16), np.linspace(0, height, 16))
w = np.ones(x.shape)
p1s = np.stack([x, y, w], axis=-1).reshape((-1, 3))

# external camera
R = np.eye(3)
t = np.array([1, 1, -30])
f = 5
K = np.array([
    [f,  0, width/2],
    [0, f, height/2],
    [0, 0, 1],
])
P = np.dot(K, np.hstack([R, t[:, None]]))

def homogenous(euclidian):
    return np.vstack((euclidian, np.ones((1, euclidian.shape[1]))))

def stroke_line(canvas, p1, p2):
    x1, y1 = p1[0] / p1[2], p1[1] / p1[2]
    x2, y2 = p2[0] / p2[2], p2[1] / p2[2]    
    canvas.stroke_line(x1, y1, x2, y2)

def stroke_lines(canvas, P, lines):
    # split into from and to coordinates per line
    p1s, p2s = np.split(lines, 2, axis=1)
    p1s = np.squeeze(p1s, axis=1)
    p2s = np.squeeze(p2s, axis=1)
    # project with camera
    pp1s = np.dot(P, homogenous(p1s.T)).T
    pp2s = np.dot(P, homogenous(p2s.T)).T

    with hold_canvas(canvas):
        for p1, p2 in zip(pp1s, pp2s):
            stroke_line(canvas, p1, p2)

    
def update_focal(f):
    f = 12
    K = np.array([
        [f,  0, width/2],
        [0, f, height/2],
        [0, 0, 1],
    ])
    normals = np.dot(np.linalg.inv(K), p1s.T).T
    p2s = p1s + normals * 10
    lines = np.stack([p1s, p2s], axis=1)  # same shape as matplotlib

    canvas.clear()
    stroke_lines(canvas, P, lines)
    

interact(update_focal, f=(0.1, 20))
canvas


interactive(children=(FloatSlider(value=10.049999999999999, description='f', max=20.0, min=0.1), Output()), _d…

RoughCanvas()